In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', None)

In [2]:
import os
import urllib.parse

In [3]:
import requests
import brotli
from bs4 import BeautifulSoup

In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

In [5]:
filein_path = "../../../data/data_wikis/1 - politician data for test scraping/"

In [6]:
fileout_path = "../../../data/data_wikis/2.1 - sk namuwiki scraping test/"

In [7]:
logfile_name = "_seleniumtest_log.txt"

# import sk & tw politician test data

In [8]:
sktest = pd.read_excel(filein_path+"sk_testset.xlsx")

In [9]:
sktest.shape

(9960, 24)

In [36]:
# sktest["URL_NamuWiki"]=np.nan

In [39]:
sktest[3000:3100]

,Name_Native,Name_Chinese,Birthdate,Sex,Age,ElectionCategory_Native,ElectionDate,Position,L1_Division,L2_Division,...,CandidateStatus,WonElection,VoteCount,VotePercent,IncumbentStatus,Occupation,Education,Career1,Career2,URL_NamuWiki
3000,문병원,文炳元,19570507,남,57,광역의원비례대표선거,20140604,Party List Legislator,울산광역시,NaN,...,등록,1,NaN,NaN,NaN,사회복지,동국대 사회과학대학원 졸업(사회복지학),(현)밝은미래복지재단 이사,(전)제18대 대선 새누리당 울산선대위 선거대책위원장,NaN
3001,문병호,文炳浩,19591227,남,56,국회의원선거,20160413,Legislator,인천광역시,부평구,...,등록,0,NaN,NaN,NaN,국회의원,서울대학교 법과대학 법학과 졸업,(현)변호사(민변),(현)국민의당 인천광역시당위원장,NaN
3002,문병화,文炳華,19620404,남,52,구·시·군의회의원선거,20140604,Legislator,경기도,하남시,...,등록,0,NaN,NaN,NaN,회사원,전신서해공업전문대학(현 호원대학교) 토목과 졸업 (2년),(현) 홍운건설 이사,(현) 하남시문화원 이사,NaN
3003,문병훈,文柄勳,19790315,남,35,구·시·군의회의원선거,20140604,Legislator,서울특별시,서초구,...,등록,1,8385.0,34.17,NaN,연구원,한양대학교 도시대학원 석사과정 졸업(도시학석사),(전)경기개발연구원 도시지역계획연구부 연구원,(현)한국산지보전협회 선임연구원,NaN
3004,문보현,文普玄,19650519,남,50,국회의원선거,20160413,Legislator,전라남도,목포시,...,등록,0,NaN,NaN,NaN,정당인,나고야대학대학원 법학박사과정수료(2000.4.1~2006.3.31.),(전)목포대학교 총학생회장,(현)정의당 정책연구위원,NaN
3005,문복란,文福蘭,19560601,여,58,구·시·군의회의원선거,20140604,Legislator,서울특별시,성동구,...,등록,1,6902.0,19.15,NaN,정당인,한국방송통신대학교 교육학과 졸업,(현) 새정치민주연합 서울특별시당 성범죄예방특별위원회 부위원장,(현) 대한적십자사 서울특별시지사 성동구후원회 사무국장,NaN
3006,문봉도,文奉道,19650505,남,49,구·시·군의회의원선거,20140604,Legislator,경상남도,의령군,...,등록,1,1293.0,33.75,NaN,의령군의회의원,한국방송통신대학교 행정학과 졸업,(현)제6대 의령군의회의원,(현)제6대 의령군의회 상·하반기 산업건설위원장,NaN
3007,문봉선,文峰仙,19620310,여,52,구·시·군의회의원선거,20140604,Legislator,경기도,과천시,...,등록,1,4713.0,26.36,NaN,정당인,동국대학교 문화예술대학원 졸업(문학 석사),(전)새누리당 과천의왕 당협 문화예술분과위원장,(현)한국문인협회 과천지부 상임 부회장,NaN
3008,문상모,文鋿模,19690202,남,45,시·도의회의원선거,20140604,Legislator,서울특별시,노원구,...,등록,1,22270.0,53.83,NaN,서울특별시의회 의원,광운대학교 상담복지정책대학원 사회복지학과 재학,(현)서울특별시의회 문화체육관광위원회 위원,(현)2018 평창동계올림픽지원 및 서울시스포츠 활성화를 위한 특별위원장,NaN
3009,문상선,文相宣,19670404,남,47,구·시·군의회의원선거,20140604,Legislator,경기도,의정부시,...,등록,0,NaN,NaN,NaN,정당인,한국방송통신대학교 경영학과 1학년 제적,(전) 민주당 국회의원 김진표 정책특보,(현) 의정부시 청소년지도협의회 사무국장,NaN


In [10]:
# percent duplicates

1 - (sktest["Name_Native"].drop_duplicates().shape[0] / sktest.shape[0])

0.12550200803212852

# scrape namuwiki with selenium

In [11]:
url_stem = "https://namu.wiki"

p_error_name = "data-v-1ac09f2e"
p_error_text = "해당 문서를 찾을 수 없습니다."

In [61]:
# instantiate selenium webdriver
service = Service(executable_path=ChromeDriverManager().install())

# scrape namuwiki using selenium
for index,p in sktest.iterrows():
    
    if(pd.isnull(p.URL_NamuWiki)):
    
        # construct url - encode non-ASCII search terms as ASCII
        url= url_stem+"/w/"+urllib.parse.quote(p.Name_Native.encode("utf-8"))

        # reset content & has_content_flag
        has_content_flag = True
        content=""

        try:

            # request url using selenium webdriver
            driver = webdriver.Chrome(service=service)
            driver.get(url)

            # check if page has content
            elements = driver.find_elements(By.TAG_NAME, 'p')

            for e in elements:
                if(e.text==p_error_text):
                    has_content_flag = False

            if(has_content_flag):
                content = driver.page_source

            # close webdriver
            driver.quit()

            # save content
            if(has_content_flag):

                # save output as an html file
                fout = open(fileout_path+"seleniumtest_"+str(index)+"_"+p.Name_Native+".html","w",encoding="utf-8")
                fout.write(content)
                fout.close()

                # update HasNamuWiki content field in dataframe
                #sktest.loc[sktest.Name_Native == p.Name_Native,'URL_NamuWiki']=url

                # save result to log file
                log_output = ",".join([str(index),p.Name_Native,url,"\n"])
                fout = open(fileout_path+logfile_name,"a",encoding="utf-8")
                fout.write(log_output)
                fout.close()

                print(index,p.Name_Native,url,"Number of <p> elements:",len(elements),"Has Content")

            else:

                # update HasNamuWiki content field in dataframe
                # sktest.loc[sktest.Name_Native == p.Name_Native,'URL_NamuWiki']=False

                # save result to log file
                log_output = ",".join([str(index),p.Name_Native,"False","\n"])
                fout = open(fileout_path+logfile_name,"a",encoding="utf-8")
                fout.write(log_output)
                fout.close()

                print(index,p.Name_Native,url,"Number of <p> elements:",len(elements),"Empty")
            
        except KeyboardInterrupt:
        
            print("Keyboard Interrupt")
            
            break

        except Exception as e:

            print(index,p.Name_Native,url,"Number of <p> elements:",len(elements),"Error")
            print(e)





3108 민태찬 https://namu.wiki/w/%EB%AF%BC%ED%83%9C%EC%B0%AC Number of <p> elements: 5 Empty
3109 민형배 https://namu.wiki/w/%EB%AF%BC%ED%98%95%EB%B0%B0 Number of <p> elements: 4 Has Content
3110 민홍철 https://namu.wiki/w/%EB%AF%BC%ED%99%8D%EC%B2%A0 Number of <p> elements: 4 Has Content
3111 박견목 https://namu.wiki/w/%EB%B0%95%EA%B2%AC%EB%AA%A9 Number of <p> elements: 5 Empty
3112 박겸수 https://namu.wiki/w/%EB%B0%95%EA%B2%B8%EC%88%98 Number of <p> elements: 4 Has Content
3113 박경미 https://namu.wiki/w/%EB%B0%95%EA%B2%BD%EB%AF%B8 Number of <p> elements: 4 Has Content
3114 박경민 https://namu.wiki/w/%EB%B0%95%EA%B2%BD%EB%AF%BC Number of <p> elements: 4 Has Content
3115 박경석 https://namu.wiki/w/%EB%B0%95%EA%B2%BD%EC%84%9D Number of <p> elements: 4 Has Content
3116 박경수 https://namu.wiki/w/%EB%B0%95%EA%B2%BD%EC%88%98 Number of <p> elements: 4 Has Content
3117 박경숙 https://namu.wiki/w/%EB%B0%95%EA%B2%BD%EC%88%99 Number of <p> elements: 4 Has Content
3118 박경식 https://namu.wiki/w/%EB%B0%95%EA%B2%BD%EC%8B%9D Numb

3197 박병만 https://namu.wiki/w/%EB%B0%95%EB%B3%91%EB%A7%8C Number of <p> elements: 5 Empty
3198 박병석 https://namu.wiki/w/%EB%B0%95%EB%B3%91%EC%84%9D Number of <p> elements: 4 Has Content
3199 박병석 https://namu.wiki/w/%EB%B0%95%EB%B3%91%EC%84%9D Number of <p> elements: 4 Has Content
3200 박병영 https://namu.wiki/w/%EB%B0%95%EB%B3%91%EC%98%81 Number of <p> elements: 5 Empty
3201 박병일 https://namu.wiki/w/%EB%B0%95%EB%B3%91%EC%9D%BC Number of <p> elements: 4 Has Content
3202 박병종 https://namu.wiki/w/%EB%B0%95%EB%B3%91%EC%A2%85 Number of <p> elements: 4 Has Content
3203 박병진 https://namu.wiki/w/%EB%B0%95%EB%B3%91%EC%A7%84 Number of <p> elements: 4 Has Content
3204 박병철 https://namu.wiki/w/%EB%B0%95%EB%B3%91%EC%B2%A0 Number of <p> elements: 4 Has Content
3205 박병훈 https://namu.wiki/w/%EB%B0%95%EB%B3%91%ED%9B%88 Number of <p> elements: 4 Has Content
3206 박보근 https://namu.wiki/w/%EB%B0%95%EB%B3%B4%EA%B7%BC Number of <p> elements: 5 Empty
3207 박보생 https://namu.wiki/w/%EB%B0%95%EB%B3%B4%EC%83%9D Number of <

In [62]:
sktest[sktest.Name_Native=="강민국"]

,Name_Native,Name_Chinese,Birthdate,Sex,Age,ElectionCategory_Native,ElectionDate,Position,L1_Division,L2_Division,...,CandidateStatus,WonElection,VoteCount,VotePercent,IncumbentStatus,Occupation,Education,Career1,Career2,URL_NamuWiki
60,강민국,姜旻局,19710303,남,43,시·도의회의원선거,20140604,Legislator,경상남도,진주시,...,등록,1,28248.0,75.31,NaN,정당인,경남대학교 대학원 졸업(법학박사),(전)경상남도 도지사 비서실장,(전)경상남도 정무보좌역,https://namu.wiki/w/%EA%B0%95%EB%AF%BC%EA%B5%AD


In [76]:
# things to check

# 0. drop duplicates. some duplicate pages are different (at least size differs). why?  

# 0.1 drop duplicates (by=[Name_Native, Name_Chinese, Birthdate]) in TW dataset, before scraping

# 0.2 add local legislators back in? but mark them as "skip"?

# 1. saved links are valid (no false positives); False links actually don't exist (no false negatives)

# 2. disambiguation? what about names that link to more than one person?
#
#       - disambiguation links are usually included in the page
#
#       - redirect from the page of a more popular person
#               https://namu.wiki/w/%EA%B0%95%EB%AF%BC%EA%B5%AD - 강민국
#               정치인에 대한 내용은 강민국(정치인) 문서를 참고하십시오.
#               https://namu.wiki/w/%EA%B0%95%EB%AF%BC%EA%B5%AD(%EC%A0%95%EC%B9%98%EC%9D%B8) - 강민국(정치인)
#
#       - redirects from a page listing links to multiple people having the same name
#               https://namu.wiki/w/%EA%B9%80%EC%9A%A9%EC%A7%84

# update sktest with selenium log from latest run

In [64]:
log_columns = ["index","Name_Native","url","dropme"]
log = pd.read_csv(fileout_path+logfile_name,header=None,encoding="utf-8",names=log_columns)
log.drop(columns=["dropme"],inplace=True)
log

,index,Name_Native,url
0,3108,민태찬,False
1,3109,민형배,https://namu.wiki/w/%EB%AF%BC%ED%98%95%EB%B0%B0
2,3110,민홍철,https://namu.wiki/w/%EB%AF%BC%ED%99%8D%EC%B2%A0
3,3111,박견목,False
4,3112,박겸수,https://namu.wiki/w/%EB%B0%95%EA%B2%B8%EC%88%98
5,3113,박경미,https://namu.wiki/w/%EB%B0%95%EA%B2%BD%EB%AF%B8
6,3114,박경민,https://namu.wiki/w/%EB%B0%95%EA%B2%BD%EB%AF%BC
7,3115,박경석,https://namu.wiki/w/%EB%B0%95%EA%B2%BD%EC%84%9D
8,3116,박경수,https://namu.wiki/w/%EB%B0%95%EA%B2%BD%EC%88%98
9,3117,박경숙,https://namu.wiki/w/%EB%B0%95%EA%B2%BD%EC%88%99


In [65]:
sktest.shape

(6226, 24)

In [66]:
sktest_updated = sktest.merge(log,how="left",on="Name_Native")
sktest_updated.shape

(6234, 26)

In [67]:
# update URL_NamuWiki with log results
sktest_updated.loc[sktest_updated.URL_NamuWiki.isnull() & sktest_updated.url.notnull(),"URL_NamuWiki"] = sktest_updated.loc[sktest_updated.URL_NamuWiki.isnull() & sktest_updated.url.notnull(),"url"]

# confirm that changes have been made
sktest_updated.loc[sktest_updated.url.notnull(),["URL_NamuWiki","url"]]

,URL_NamuWiki,url
3108,False,False
3109,https://namu.wiki/w/%EB%AF%BC%ED%98%95%EB%B0%B0,https://namu.wiki/w/%EB%AF%BC%ED%98%95%EB%B0%B0
3110,https://namu.wiki/w/%EB%AF%BC%ED%99%8D%EC%B2%A0,https://namu.wiki/w/%EB%AF%BC%ED%99%8D%EC%B2%A0
3111,False,False
3112,https://namu.wiki/w/%EB%B0%95%EA%B2%B8%EC%88%98,https://namu.wiki/w/%EB%B0%95%EA%B2%B8%EC%88%98
3113,https://namu.wiki/w/%EB%B0%95%EA%B2%BD%EB%AF%B8,https://namu.wiki/w/%EB%B0%95%EA%B2%BD%EB%AF%B8
3114,https://namu.wiki/w/%EB%B0%95%EA%B2%BD%EB%AF%BC,https://namu.wiki/w/%EB%B0%95%EA%B2%BD%EB%AF%BC
3115,https://namu.wiki/w/%EB%B0%95%EA%B2%BD%EC%84%9D,https://namu.wiki/w/%EB%B0%95%EA%B2%BD%EC%84%9D
3116,https://namu.wiki/w/%EB%B0%95%EA%B2%BD%EC%88%98,https://namu.wiki/w/%EB%B0%95%EA%B2%BD%EC%88%98
3117,https://namu.wiki/w/%EB%B0%95%EA%B2%BD%EC%88%99,https://namu.wiki/w/%EB%B0%95%EA%B2%BD%EC%88%99


In [68]:
# change "False" strings to boolean False
sktest_updated.loc[sktest_updated.URL_NamuWiki=="False","URL_NamuWiki"]=False

In [69]:
sktest_updated[sktest_updated["index"].notnull()]

,Name_Native,Name_Chinese,Birthdate,Sex,Age,ElectionCategory_Native,ElectionDate,Position,L1_Division,L2_Division,...,VoteCount,VotePercent,IncumbentStatus,Occupation,Education,Career1,Career2,URL_NamuWiki,index,url
3108,민태찬,閔太纘,19640902,남,49,시·도의회의원선거,20140604,Legislator,충청북도,진천군,...,NaN,NaN,NaN,자영업,성균관대학교 일반대학원 신문방송학과 졸업 (정치학 석사),(전)동양일보 기자,(전)아시아뉴스통신 기자,False,3108.0,False
3109,민형배,閔炯培,19610615,남,52,구·시·군의 장선거,20140604,Chief,광주광역시,광산구,...,127437.0,83.36,NaN,자치구청장,전남대학교 일반대학원 사회학박사,(현)광산구청장,(전)노무현대통령 청와대 비서관,https://namu.wiki/w/%EB%AF%BC%ED%98%95%EB%B0%B0,3109.0,https://namu.wiki/w/%EB%AF%BC%ED%98%95%EB%B0%B0
3110,민홍철,閔洪喆,19610418,남,54,국회의원선거,20160413,Legislator,경상남도,김해시,...,67866.0,55.96,NaN,국회의원,부산대학교 대학원 법학과 졸업(법학석사),(전)고등군사법원 법원장(육군준장),(현) 제19대 국회의원,https://namu.wiki/w/%EB%AF%BC%ED%99%8D%EC%B2%A0,3110.0,https://namu.wiki/w/%EB%AF%BC%ED%99%8D%EC%B2%A0
3111,박견목,朴堅穆,19610513,남,54,국회의원선거,20160413,Legislator,부산광역시,기장군,...,NaN,NaN,NaN,부경대학교 방위과학기술연구소 계약교수,수원대학교 행정대학원 졸업(행정학석사),(전)국군기무사령부 육군 준장,(전)여의도연구원 정책자문위원,False,3111.0,False
3112,박겸수,朴謙洙,19591026,남,54,구·시·군의 장선거,20140604,Chief,서울특별시,강북구,...,79901.0,52.34,NaN,정무직 공무원,한양대학교 대학원 행정학과 박사과정 수료,(현)강북구청장,(전)서울특별시의회의원,https://namu.wiki/w/%EB%B0%95%EA%B2%B8%EC%88%98,3112.0,https://namu.wiki/w/%EB%B0%95%EA%B2%B8%EC%88%98
3113,박경미,朴炅美,19651015,여,50,비례대표국회의원선거,20160413,Party List Legislator,전국,전국,...,NaN,NaN,NaN,교수,미국 일리노이대학교 수학교육학 박사(1989.8. ~ 1993.1.),(현)홍익대학교 수학교육과 교수,(현)대학수학교육학회 이사,https://namu.wiki/w/%EB%B0%95%EA%B2%BD%EB%AF%B8,3113.0,https://namu.wiki/w/%EB%B0%95%EA%B2%BD%EB%AF%B8
3114,박경민,朴景旻,19751205,남,40,국회의원선거,20160413,Legislator,부산광역시,사하구,...,NaN,NaN,NaN,통닭배달,동아대학교 산업디자인학과 졸업,(현)쌈지휴게소 대표,(현)헌혈58회 조혈모세포기증,https://namu.wiki/w/%EB%B0%95%EA%B2%BD%EB%AF%BC,3114.0,https://namu.wiki/w/%EB%B0%95%EA%B2%BD%EB%AF%BC
3115,박경석,朴京奭,19690214,남,45,시·도의회의원선거,20140604,Legislator,부산광역시,동래구,...,NaN,NaN,NaN,금융업(투자자문업),부산대학교 사회복지학과 졸업,(현) (주)밸류인베스트코리아 수석팀장,(전) 부산환경운동연합 운영위원,https://namu.wiki/w/%EB%B0%95%EA%B2%BD%EC%84%9D,3115.0,https://namu.wiki/w/%EB%B0%95%EA%B2%BD%EC%84%9D
3116,박경수,朴京守,19621022,여,51,기초의원비례대표선거,20140604,Party List Legislator,경기도,양주시,...,NaN,NaN,NaN,회사원,광운대학교 상담복지정책대학원 졸업(사회복지학 석사),(전)양주시사회복지사협회회장,(전)양주시백석읍주민자치부위원장,https://namu.wiki/w/%EB%B0%95%EA%B2%BD%EC%88%98,3116.0,https://namu.wiki/w/%EB%B0%95%EA%B2%BD%EC%88%98
3117,박경숙,朴瓊淑,19610715,여,52,기초의원비례대표선거,20140604,Party List Legislator,충청북도,보은군,...,NaN,NaN,NaN,정당인,청주대학교사범대학 지리교육과 졸업,(전)보은중 자모회장,(현)파랑새 지역아동센터 운영위원,https://namu.wiki/w/%EB%B0%95%EA%B2%BD%EC%88%99,3117.0,https://namu.wiki/w/%EB%B0%95%EA%B2%BD%EC%88%99


In [70]:
# drop log column (url)
sktest_updated.drop(columns=["index","url"],inplace=True)

In [71]:
# drop duplicate rows
sktest_updated.drop_duplicates().shape

(6226, 24)

In [72]:
# drop duplicate politicians
sktest_updated.sort_values(["Name_Native","Name_Chinese","Birthdate","ElectionDate"],inplace=True)
sktest_updated.drop_duplicates(["Name_Native","Name_Chinese","Birthdate"],keep="last",inplace=True)

In [73]:
sktest_updated.shape

(6226, 24)

In [74]:
# refresh and save updated sktest
sktest = sktest_updated
sktest.to_excel(filein_path+"sk_testset.xlsx",index=False)

In [75]:
# delete log file
os.remove(fileout_path+logfile_name)

In [33]:
# DEPRECATED - assign URL_NamuWiki link based on files already scraped - I needed this before I created a log file

# import os
# filelist = os.listdir(fileout_path)
# test = sktest
# url_stem = "https://namu.wiki"
# test[100:200]
# scraped = [f.split("_")[2].split(".")[0].strip() for f in filelist]
# for s in scraped:
#     url = url_stem+"/w/"+urllib.parse.quote(s.encode("utf-8"))
#     # print(url_stem+"/w/"+urllib.parse.quote(s.encode("utf-8")))
#     test.loc[test.Name_Native==s,"URL_NamuWiki"]=url
# test[100:200]

In [45]:
# DEPRECATED - drop some local legislators, in order to speed things up

# s_url_notnull = sktest[sktest.URL_NamuWiki.notnull()]
# s_url_isnull = sktest[sktest.URL_NamuWiki.isnull()]
# s_url_isnull_notlocalleg = s_url_isnull[s_url_isnull.ElectionCategory_Native!="구·시·군의회의원선거"]
# s_url_isnull_localleg = s_null[s_url_isnull.ElectionCategory_Native=="구·시·군의회의원선거"]
# sktest_fewer_localleg = pd.concat([s_url_notnull,s_url_isnull_notlocalleg],ignore_index=True)
# sktest_fewer_localleg.groupby(["ElectionDate","ElectionCategory_Native"]).agg({"Name_Native":"count"}).sort_values("ElectionDate",ascending=False)